In [ ]:
import pandas as pd
import hopsworks
import joblib
import datetime
from PIL import Image
from datetime import datetime
import dataframe_image as dfi
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot
import seaborn as sns
import requests
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()
api_key = os.getenv("HOPSWORKS_API_KEY")

In [ ]:
project = hopsworks.login(api_key_value=api_key, project="DD2223_lab1")
fs = project.get_feature_store()

In [ ]:
mr = project.get_model_registry()
model = mr.get_model("winequality_model", version=1)
model_dir = model.download()
model = joblib.load(model_dir + "/winequality_model.pkl")

In [ ]:
feature_view = fs.get_feature_view(name="wine_quality", version=1)
batch_data = feature_view.get_batch_data()

In [ ]:
y_pred = model.predict(batch_data)
#print(y_pred)
offset = 1
wine = y_pred[y_pred.size-offset]
wine_url = "https://raw.githubusercontent.com/featurestoreorg/serverless-ml-course/main/src/01-module/assets/" + wine + ".png"
print("Predicted wine quality: " + wine)
img = Image.open(requests.get(wine_url, stream=True).raw)            
img.save("./latest_iris.png")
dataset_api = project.get_dataset_api()    
dataset_api.upload("./latest_iris.png", "Resources/images", overwrite=True)

In [ ]:
wine_fg = fs.get_feature_group(name="wine_quality", version=1)
df = wine_fg.read() 
#print(df)
label = df.iloc[-offset]["variety"]
label_url = "https://raw.githubusercontent.com/featurestoreorg/serverless-ml-course/main/src/01-module/assets/" + label + ".png"
print("Actual wine quality: " + label)
img = Image.open(requests.get(label_url, stream=True).raw)            
img.save("./actual_iris.png")
dataset_api.upload("./actual_iris.png", "Resources/images", overwrite=True)

In [ ]:
monitor_fg = fs.get_or_create_feature_group(name="wine_quality_predictions",
                                            version=1,
                                            primary_key=["datetime"],
                                            description="Wine Quality Prediction/Outcome Monitoring"
                                            )

now = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
data = {
    'prediction': [wine],
    'label': [label],
    'datetime': [now],
    }
monitor_df = pd.DataFrame(data)
monitor_fg.insert(monitor_df, write_options={"wait_for_job" : False})

history_df = monitor_fg.read()
# Add our prediction to the history, as the history_df won't have it - 
# the insertion was done asynchronously, so it will take ~1 min to land on App
history_df = pd.concat([history_df, monitor_df])

In [ ]:
df_recent = history_df.tail(4)
dfi.export(df_recent, './df_recent.png', table_conversion = 'matplotlib')
dataset_api.upload("./df_recent.png", "Resources/images", overwrite=True)

predictions = history_df[['prediction']]
labels = history_df[['label']]

# Only create the confusion matrix when our wine_quality_predictions feature group has examples of all 7 qualities
print("Number of different wine quality predictions to date: " + str(predictions.value_counts().count()))
if predictions.value_counts().count() == 3:
    results = confusion_matrix(labels, predictions)

    df_cm = pd.DataFrame(results, ['True Setosa', 'True Versicolor', 'True Virginica'],
                            ['Pred Setosa', 'Pred Versicolor', 'Pred Virginica'])

    cm = sns.heatmap(df_cm, annot=True)
    fig = cm.get_figure()
    fig.savefig("./confusion_matrix.png")
    dataset_api.upload("./confusion_matrix.png", "Resources/images", overwrite=True)
else:
    print("You need 7 different wine quality predictions to create the confusion matrix.")
    print("Run the batch inference pipeline more times until you get 7 different wine quality predictions") 